In [1]:
import pandas as pd
from sqlalchemy import create_engine, text

DB_USER = "root"        
DB_PASS = "03091969Aleman"  
DB_HOST = "localhost"
DB_PORT = 3306
DB_NAME = "sakila"

CONN_STR = f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(CONN_STR, pool_pre_ping=True)

# Test
with engine.connect() as conn:
    print(pd.read_sql(text("SELECT 1 AS ok"), conn))

   ok
0   1


In [2]:
def rentals_month(engine, month: int, year: int) -> pd.DataFrame:
    query = text("""
        SELECT rental_id, rental_date, inventory_id, customer_id, return_date, staff_id, last_update
        FROM rental
        WHERE YEAR(rental_date) = :year AND MONTH(rental_date) = :month
        ORDER BY rental_date
    """)
    with engine.connect() as conn:
        return pd.read_sql(query, conn, params={"year": year, "month": month})

In [3]:
def rental_count_month(df: pd.DataFrame, month: int, year: int) -> pd.DataFrame:
    mm = str(month).zfill(2)
    colname = f"rentals_{mm}_{year}"
    out = (df.groupby("customer_id")["rental_id"]
             .count()
             .rename(colname)
             .reset_index()
             .sort_values([colname, "customer_id"], ascending=[False, True])
             .reset_index(drop=True))
    return out

In [4]:
def compare_rentals(df_m1: pd.DataFrame, df_m2: pd.DataFrame) -> pd.DataFrame:
    c1 = [c for c in df_m1.columns if c != 'customer_id'][0]
    c2 = [c for c in df_m2.columns if c != 'customer_id'][0]
    merged = df_m1.merge(df_m2, on="customer_id", how="outer").fillna(0)
    merged[c1] = merged[c1].astype(int)
    merged[c2] = merged[c2].astype(int)
    merged["difference"] = merged[c2] - merged[c1]
    return merged.sort_values(["difference", c2, "customer_id"], ascending=[False, False, True]).reset_index(drop=True)

# Example: May vs June, 2005
m1, y1 = 5, 2005
m2, y2 = 6, 2005
df_may  = rentals_month(engine, m1, y1)
df_june = rentals_month(engine, m2, y2)
cnt_may  = rental_count_month(df_may, m1, y1)
cnt_june = rental_count_month(df_june, m2, y2)
comparison = compare_rentals(cnt_may, cnt_june)
comparison.head(10)

,customer_id,rentals_05_2005,rentals_06_2005,difference
0,31,0,11,11
1,454,1,10,9
2,329,0,9,9
3,213,1,9,8
4,295,1,9,8
5,457,1,9,8
6,178,0,8,8
7,268,0,8,8
8,336,0,8,8
9,340,0,8,8
